# Cloud Vision API (alpha) : Demo

The Cloud Vision API enables developers to integrate state-of-the-art computer vision algorithms in a line of code, without any algorithmic or integration struggle. Below is a demo of the algorithms currently available in the API, as well as example results below:

* **Super-resolution**: enhances the quality of an image <br />
* **Style Transfer**: transform a photo into a piece of art <br />
* **Semantic Segmentation**: classifies each pixel of the picture into a class such as car, pedestrian, road, etc... <br />
* **Background Segmentation**: creates a mask to separate between the foreground and background in portrait pictures <br />
* **Monocular Depth Estimation**:  estimates how far each pixel is from the camera <br />

You think that another algorithm should be included in this API? Please tell me about it at sebderhy@gmail.com.

<table>
    <tr><td><center>input image</center></td><td><center>semantic segmentation</center></td></tr>
    <tr><td><img src='test_img.png'></td><td><img src='img_out/semseg-3.png'></td></tr>
    <tr><td><center>Super-resolution</center></td><td><center>Depth</center></td></tr>
    <tr><td><img src='img_out/superres-2b.png'></td><td><img src='img_out/depth-bts.png'></td></tr>
    <tr><td><center>Style Transfer 1</center></td><td><center>Style Transfer 3</center></td></tr>
    <tr><td><img src='img_out/styletransf-1.png'></td><td><img src='img_out/styletransf-3.png'></td></tr>
    <tr><td><center>Input selfie</center></td><td><center>Background Swap</center></td></tr>
    <tr><td><img src='https://images.unsplash.com/photo-1543486958-d783bfbf7f8e?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&w=1000&q=80'></td><td><img src='img_out/binseg-3-bgswap.png'></td></tr>

</table>

## Read this before you use it

* Be patient! When you submit an image, the results may take about 20 seconds to arrive
* The *semantic segmentation and depth estimation* algorithms will work well **on road pictures** (i.e. pictures taken from a car), because they have been trained on such datasets.
* The Background segmentation algorithm will work **only on portraits/selfies**, and is currently **only giving a rough contour** (typically, it will miss the subtilities in hair)
* Keep in mind that this is a side-project and not a finished product yet! Although I do my best to keep everything working and resilient, the results may be disappointing, and the server may fail (apologies if that's the case). In any case, please share your feedback with me (sebderhy@gmail.com), so that I can improve it accoridingly.   


In [3]:
from utils import *

In [4]:
from ipywidgets import widgets

In [5]:
URL_BG = 'https://img.theculturetrip.com/768x432/wp-content/uploads/2018/01/webp-net-compress-image-45.jpg'
def viz_out(model_name, img_in, img_out, url_bg=URL_BG):
    if model_name.startswith('binseg'):
        response = requests.get(url_bg)
        bg = Image.open(BytesIO(response.content))
        bg2 = bg.resize(img_in.size)
        img_out2=Image.composite(img_in, bg2, img_out)
    else:
        img_out2 = img_out
    return img_out2

## Option 1: call from URL 

Define the Image URL you want to test below, and click on process. Both the input and output will be displayed. 

**Be patient, the results can take ~20 seconds to appear**

In [6]:
url_placeholder = widgets.Text(
    placeholder='URL of an image',
    value = 'https://www.go-telaviv.com/images/driving-in-israel-tel-aviv-traffic-jam1.jpg',
    disabled=False
)

In [7]:
out_pl_url = widgets.Output()
out_pl_url.clear_output()

In [8]:
out_pl_url_2 = widgets.Output()
out_pl_url_2.clear_output()

In [9]:
models_list_url = widgets.Dropdown(
    options=[('Super-resolution', 'superres-2b'),
             ('Style Transfer 1', 'styletransf-1'),
             ('Style Transfer 2', 'styletransf-2'),
             ('Style Transfer 3', 'styletransf-3'),
             ('Semantic Segmentation', 'semseg-3'),
             ('Depth', 'depth-bts'),
             ('Background Extraction', 'binseg-3')],
    value='semseg-3',
    disabled=False,
)

In [21]:
btn_run_url = widgets.Button(description='Process')

In [22]:
lbl_status_url = widgets.Label()
lbl_status_url.value = 'waiting for user input'

In [24]:
def on_click_process_url(change):
    lbl_status_url.value = 'loading input image...'
    response = requests.get(url_placeholder.value)
    img = Image.open(BytesIO(response.content))
    lbl_status_url.value = 'Image loaded (see below). Processing...'
    out_pl_url.clear_output()
    with out_pl_url: display(img)
    r = URLImgAPICall(url_placeholder.value, f'urlimg2img/{models_list_url.value}/')
    img_out = response2img(r)
    img_out = viz_out(models_list_url.value, img, img_out)
    out_pl_url_2.clear_output()
    with out_pl_url_2: display(img_out)
    lbl_status_url.value = 'Here is the output image !'

btn_run_url.on_click(on_click_process_url)

In [15]:
widgets.VBox([widgets.Label('Choose your algorithm'), models_list_url,
              widgets.Label('Write your image URL'), url_placeholder, 
              widgets.Label('Click below to process the image'), btn_run_url, 
              lbl_status_url, out_pl_url, out_pl_url_2])

Elapsed time: 4.7238757610321045


## Option 2: Import local picture

In [18]:
btn_upload = widgets.FileUpload()

In [19]:
out_pl = widgets.Output()
out_pl.clear_output()

In [20]:
out_pl2 = widgets.Output()
out_pl2.clear_output()

In [21]:
models_list = widgets.Dropdown(
    options=[('Super-resolution', 'superres-2b'),
             ('Style Transfer 1', 'styletransf-1'),
             ('Style Transfer 2', 'styletransf-2'),
             ('Style Transfer 3', 'styletransf-3'),
             ('Semantic Segmentation', 'semseg-3'),
             ('Depth', 'depth-bts'),
             ('Background Extraction', 'binseg-3')],
    value='styletransf-1',
    disabled=False,
)

In [22]:
btn_run = widgets.Button(description='Process')

In [23]:
lbl_status = widgets.Label()
lbl_status.value = 'waiting for user input'

In [24]:
def on_click_process(change):
    lbl_status.value = 'loading input image...'
    img = Image.open(BytesIO(btn_upload.data[-1]))
    out_pl.clear_output()
    with out_pl: display(img)
    lbl_status_url.value = 'Image loaded (see below). Processing...'
    r = pilImgAPICall(img, f'img2img/{models_list.value}/')
    img_out = response2img(r)
    img_out = viz_out(models_list.value, img, img_out)
    out_pl2.clear_output()
    with out_pl2: display(img_out)
    lbl_status.value = 'Here is the output image !'

btn_run.on_click(on_click_process)

Upload your image below, and click on process. Both the input and output will be displayed. 

**Be patient, the results can take ~20 seconds to appear**

In [25]:
widgets.VBox([widgets.Label('Choose your algorithm'), models_list,
              widgets.Label('Choose your image'), btn_upload, 
              widgets.Label('Click below to process the image'), btn_run, 
              lbl_status, out_pl, out_pl2])

Elapsed time: 12.254088640213013
